Importing necessary libraries.

In [1]:
from __future__ import print_function
import tensorflow as tf
from tensorflow.contrib import rnn
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


Defining Parameters

In [2]:
learning_rate = 0.001
training_steps = 10000
batch_size = 128
display_step = 200
ninput = 28 
timesteps = 28 
hidden = 128 
classes = 10 

X = tf.placeholder("float", [None, timesteps, ninput])
Y = tf.placeholder("float", [None, classes])

w = {'out': tf.Variable(tf.random_normal([hidden, classes]))}
b = {'out': tf.Variable(tf.random_normal([classes]))}

Defining the LSTM Cell

In [3]:
def RNN(x, w, b):
    x = tf.unstack(x, timesteps, 1)
    lstm_cell = rnn.BasicLSTMCell(hidden, forget_bias=1.0)
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    return tf.matmul(outputs[-1], w['out']) + b['out']

Graph Model Creation using Adam Optimizer

In [4]:
lg = RNN(X, w, b)
pred = tf.nn.softmax(lg)
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=lg, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate,beta1=0.9,beta2=0.999,epsilon=1e-08,use_locking=False,name='Adam')
train_op = optimizer.minimize(loss_op)
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

Initializing the variables

In [5]:
init = tf.global_variables_initializer()

Running the training and prediction

In [6]:
with tf.Session() as sess:

    sess.run(init)
    for step in range(1, training_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        batch_x = batch_x.reshape((batch_size, timesteps, ninput))
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
    
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + "{:.4f}".format(loss) + ", Training Accuracy= " + "{:.3f}".format(acc))

    print("Completed")

    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, timesteps, ninput))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))

Step 1, Minibatch Loss= 2.4018, Training Accuracy= 0.094
Step 200, Minibatch Loss= 0.3202, Training Accuracy= 0.914
Step 400, Minibatch Loss= 0.1770, Training Accuracy= 0.961
Step 600, Minibatch Loss= 0.1061, Training Accuracy= 0.969
Step 800, Minibatch Loss= 0.0440, Training Accuracy= 0.992
Step 1000, Minibatch Loss= 0.0330, Training Accuracy= 0.992
Step 1200, Minibatch Loss= 0.0705, Training Accuracy= 0.977
Step 1400, Minibatch Loss= 0.0862, Training Accuracy= 0.977
Step 1600, Minibatch Loss= 0.0430, Training Accuracy= 0.984
Step 1800, Minibatch Loss= 0.0144, Training Accuracy= 1.000
Step 2000, Minibatch Loss= 0.0192, Training Accuracy= 0.992
Step 2200, Minibatch Loss= 0.0299, Training Accuracy= 0.984
Step 2400, Minibatch Loss= 0.0135, Training Accuracy= 1.000
Step 2600, Minibatch Loss= 0.1517, Training Accuracy= 0.961
Step 2800, Minibatch Loss= 0.0108, Training Accuracy= 1.000
Step 3000, Minibatch Loss= 0.0487, Training Accuracy= 0.977
Step 3200, Minibatch Loss= 0.0321, Training Acc

Final Output : Training Accuracy - 1.00, Test Accuracy - 0.992